In [69]:
from sklearn.model_selection import train_test_split
import data_preprocessing.telco as telco_data
import lib.regression as reg
from sklearn.metrics import accuracy_score
from numpy.random import choice
import numpy as np

import importlib
importlib.reload(reg)
importlib.reload(telco_data)

<module 'data_preprocessing.telco' from '/mnt/604a8326-53f8-4a01-bed2-313518eb2318/LinuxData/Documents/Git/assignments/adaboost/data_preprocessing/telco.py'>

In [70]:
(X, Y) = telco_data.load(column_count=10)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
X.shape

Load telco dataset


(7043, 11)

In [71]:
K_ensembles = 10 # number of hypotheses
N_dataset_size = X_train.shape[0]
W_data_weights = np.full(N_dataset_size, 1 / N_dataset_size)
Z_model_weights = np.zeros((K_ensembles,))
models = []

In [72]:
print(Z_model_weights[0])
Z_model_weights.shape

0.0


(10,)

In [73]:
print(W_data_weights[0])
W_data_weights.shape

0.0001774937877174299


(5634,)

In [74]:
for k in range(K_ensembles):
    sample_indices = choice(range(N_dataset_size), N_dataset_size,
              p=W_data_weights)
    X_sample_train = X_train[sample_indices]
    Y_sample_train = y_train[sample_indices]

    logistic_regression = reg.LogisticRegression()
    logistic_regression.fit(X_sample_train,
                            Y_sample_train,
                            number_of_epochs=10000,
                            learning_rate=0.01)

    print(f"Train accuracy of model {k}: {accuracy_score(y_train,logistic_regression.predict(X_train))}")

    print(f"Test accuracy of model {k}: {accuracy_score(y_test, logistic_regression.predict(X_test))}")
    models.append(logistic_regression)
    y_hypothesis = logistic_regression.predict(X_train)
    error = 0
    for j in range(N_dataset_size):
        if not y_hypothesis[j] == y_train[j]:
            error += W_data_weights[j]
    print(f"error: {error}")
    if error > 0.5:
        print("skipping ensemble")
        continue

    for j in range(N_dataset_size):
        if y_hypothesis[j] == y_train[j]:
            W_data_weights[j] += ((W_data_weights[j]*error)/(1-error))

    W_data_weights = W_data_weights / np.sum(W_data_weights)
    Z_model_weights[k] = np.log((1-error)/error)

Z_model_weights

# loss = logistic_regression.fit(X_train, y_train, batch_size=100, number_of_epochs=1000, learning_rate=0.01)

Train accuracy of model 0: 0.7541711040113596
Test accuracy of model 0: 0.7551454932576295
error: 0.24582889598863444
Train accuracy of model 1: 0.7479588214412496
Test accuracy of model 1: 0.7579843860894251
error: 0.20899531316186004
Train accuracy of model 2: 0.7563010294639687
Test accuracy of model 2: 0.7579843860894251
error: 0.1676773422953691
Train accuracy of model 3: 0.7554135605253817
Test accuracy of model 3: 0.7601135557132718
error: 0.14553556658471417
Train accuracy of model 4: 0.7534611288604899
Test accuracy of model 4: 0.7572746628814763
error: 0.1287094840189294
Train accuracy of model 5: 0.7575434859779908
Test accuracy of model 5: 0.7615330021291696
error: 0.11114164243643772
Train accuracy of model 6: 0.7545260915867945
Test accuracy of model 6: 0.759403832505323
error: 0.10210045154363255
Train accuracy of model 7: 0.758075967341143
Test accuracy of model 7: 0.7615330021291696
error: 0.09070681060734695
Train accuracy of model 8: 0.7596734114306
Test accuracy of 

array([1.12098352, 1.33099207, 1.60217862, 1.77005438, 1.91241766,
       2.07913245, 2.17410105, 2.30503514, 2.40022896, 2.49129821])

In [75]:
weighted_predictions = np.zeros((K_ensembles, X_test.shape[0]))

for i in range(K_ensembles):
    Y_pred = models[i].predict(X_test)
    Y_pred = np.where(Y_pred == 0, -1, Y_pred) * Z_model_weights[i]
    weighted_predictions[i] = Y_pred.reshape((Y_pred.shape[0],))

prediction_sums = np.sum(weighted_predictions, axis=0)
predictions = np.where(prediction_sums > 0, 1, prediction_sums)
Y_pred = np.where(prediction_sums <= 0, 0, predictions)

print(accuracy_score(y_test, Y_pred))

0.7622427253371186
